In [111]:
import os
import sys
import pyspark
from pyspark.sql import SparkSession
from pyspark import SQLContext


In [140]:
#df = op.load.csv("C:\\Users\\Ajibola Vincent\\Documents\\Udacity\\ay1.csv")
#sqlContext = SQLContext(sc)
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

df = spark.read.csv("C:\\Users\\Ajibola Vincent\\Documents\\Udacity\\ay1.csv",header=True,sep=",");
sqlContext = SQLContext(spark)
df.count()


506

In [141]:
df = df.na.drop(subset=["account_nbr"])
#df.count()
df.count()

263

In [142]:
df.printSchema()

root
 |-- trans_id: string (nullable = true)
 |-- account_id: string (nullable = true)
 |-- account_nbr: string (nullable = true)
 |-- date: string (nullable = true)
 |-- type: string (nullable = true)
 |-- operation: string (nullable = true)
 |-- amount: string (nullable = true)
 |-- balance: string (nullable = true)
 |-- k_symbol: string (nullable = true)
 |-- bank: string (nullable = true)



In [144]:
df.select('account_id').distinct().count()

17

In [146]:
df.registerTempTable("interactions")

In [163]:
tcp_interactions = sqlContext.sql("""
    SELECT account_id, COUNT(DISTINCT account_nbr ) c FROM interactions GROUP BY account_id HAVING c >= 1""")

In [164]:
tcp_interactions.show()
#tcp.show()

+----------+---+
|account_id|  c|
+----------+---+
|        15|  1|
|        34|  3|
|        22|  1|
|        35|  3|
|        31|  1|
|        18|  1|
|      2357|  1|
|        27|  1|
|        26|  2|
|        23|  1|
|      3818|  1|
|      2177|  1|
|        33|  1|
|         9|  1|
|       485|  2|
|         2|  4|
|      1844|  3|
+----------+---+

